# Load a ome-tif object and compute mean fluorescence after background substraction

In [1]:
import os
import numpy as np
import tifffile
import matplotlib.pyplot as plt

In [13]:
main_dir = r"C:\Users\adang\OneDrive\Escritorio\ROIs_Single_Channel\20250528_M1_cells2"
experiment_name = "roi_00_ctr.ome"          # también funciona .ome.tif / .ome.tiff
ome_tif_path = os.path.join(main_dir, experiment_name)

In [14]:
file_paths = [
    os.path.join(main_dir, f)
    for f in sorted(os.listdir(main_dir))
    if f.lower().endswith(exts)
]

node_names = []                # Para elegir como antes (p. ej., node_names[0])
node_map = {}                  # label -> (path, series_index)

print("Available OME files:")
for path in file_paths:
    fname = os.path.basename(path)
    try:
        with tifffile.TiffFile(path) as tif:
            if not tif.series:
                print(f"  {fname}: no data found")
                continue

            # Imprime todas las series (si hay varias)
            for si, s in enumerate(tif.series):
                axes = s.axes       # p.ej., 'TCZYX', 'CZYX', 'ZYX'
                shape = s.shape
                # Normaliza a TCZYX con 1 en dims ausentes
                sizes = {ax: n for ax, n in zip(axes, shape)}
                tczyx = tuple(sizes.get(ax, 1) for ax in "TCZYX")

                # Etiqueta como el nombre base (si hay >1 series, añade [sN])
                base = os.path.splitext(fname)[0]
                label = base if len(tif.series) == 1 else f"{base}[s{si}]"

                node_names.append(label)
                node_map[label] = (path, si)

                # Muestra como en Zarr: nombre y tupla (T, C, Z, Y, X)
                print(f"  {label}: {tczyx}")  # e.g., (T, C, Z, Y, X)

    except tifffile.TiffFileError:
        print(f"  {fname}: no es un TIFF legible (¿solo OME-XML?).")
    except Exception as e:
        print(f"  {fname}: error al leer ({type(e).__name__}: {e})")

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'C:\\Users\\adang\\OneDrive\\Escritorio\\ROIs_Single_Channel\\20250528_M1_cells2'

In [11]:
exts = ('.ome.tif', '.ome.tiff', '.ome')
file_list = sorted([f for f in os.listdir(main_dir) if f.lower().endswith(exts)])

print("Available OME files:")
for fname in file_list:
    path = os.path.join(main_dir, fname)
    try:
        with tifffile.TiffFile(path) as tif:
            s = tif.series[0]
            axes, shape = s.axes, s.shape
            sizes = {ax: n for ax, n in zip(axes, shape)}
            tczyx = tuple(sizes.get(ax, 1) for ax in "TCZYX")
            print(f"  {fname}: axes={axes}, shape={shape} -> TCZYX={tczyx}")
    except tifffile.TiffFileError:
        print(f"  {fname}: no es TIFF legible (¿podría ser solo OME-XML?).")

Available OME files:


In [12]:
import os
import tifffile

# Ruta al OME-TIFF (usa tus variables existentes)
ome_tif_path = os.path.join(main_dir, experiment_name)

if not os.path.isfile(ome_tif_path):
    raise FileNotFoundError(f"No existe el archivo: {ome_tif_path}")

# Listar "series" (análogas a nodos) y sus formas
node_names = []  # mantenemos este nombre para compatibilidad con tu flujo
print("Available OME series:")

with tifffile.TiffFile(ome_tif_path) as tif:
    for i, s in enumerate(tif.series):
        # Nombre legible de la serie
        name = s.name if s.name else f"series_{i}"
        node_names.append(name)

        # Ejes y forma nativa reportados por tifffile (p. ej., 'TCZYX', 'CZYX', 'ZYX', etc.)
        axes = s.axes
        shape = s.shape

        # Normaliza a TCZYX para que sea comparable a tu (T, C, Z, Y, X) de Zarr
        sizes = {ax: n for ax, n in zip(axes, shape)}
        tczyx = tuple(sizes.get(ax, 1) for ax in "TCZYX")  # faltantes -> 1

        print(f"  {name}: axes={axes}, shape={shape}  ->  as TCZYX={tczyx}")

FileNotFoundError: No existe el archivo: C:\Users\ADMIN\Desktop\ROIs_refinadas\Sin_NED\Exp_20250429_Sin_NED\20250528_M1_cells2

In [4]:
# Umbral de fondo a restar (valor escalar)
background_value = 130

# Qué proyectar y cómo:
project_over = 'T'    # 'T' (tiempo) o 'Z' (profundidad)
channel_index = 0     # índice de canal a usar
z_index = 0           # usado si project_over='T' (se fija un Z)
t_range = (0, 50)     # rango de T [inicio, fin_exclusivo); None = todo

# Guardado
save_result = True
output_png = "projection_mean_bgsub.png"

In [5]:
def load_ome_tiff_as_tczyx(path):
    """
    Carga un OME-TIFF y devuelve:
      arr_tczyx: ndarray con ejes normalizados a (T, C, Z, Y, X)
      sizes: dict con tamaños {'T','C','Z','Y','X'}
    """
    with tifffile.TiffFile(path) as tif:
        series = tif.series[0]
        axes = series.axes  # e.g., 'TCZYX', 'CZYX', 'ZYX', etc.
        data = series.asarray()  # carga arreglo completo

    # Asegura que existan todas las claves en sizes
    sizes = {ax: 1 for ax in 'TCZYX'}
    for ax, n in zip(axes, data.shape):
        sizes[ax] = n

    # Reordena a TCZYX
    # Primero, expande dims faltantes en el orden original:
    arr = data
    # Si falta alguna dimensión de 'TCZYX', la insertamos al frente para mapear con moveaxis después
    # Construimos un eje virtual con el orden actual 'axes' y añadimos las que falten al final
    current_axes = list(axes)
    for ax in 'TCZYX':
        if ax not in current_axes:
            arr = np.expand_dims(arr, axis=0)
            current_axes.insert(0, ax)  # insertamos al frente (posición no crítica; luego reordenamos)
    # Ahora movemos al orden TCZYX
    src_order = ''.join(current_axes)
    perm = [src_order.index(ax) for ax in 'TCZYX']
    arr_tczyx = np.moveaxis(arr, range(arr.ndim), perm)

    # Ajusta sizes por si expandimos dims
    sizes = {ax: arr_tczyx.shape[i] for i, ax in enumerate('TCZYX')}
    return arr_tczyx, sizes


def subtract_background_and_clip(arr, bg_value):
    """
    Resta bg_value y satura negativos a 0. Devuelve float32.
    """
    arr = arr.astype(np.float32) - float(bg_value)
    np.maximum(arr, 0, out=arr)
    return arr


def mean_projection(arr_tczyx, project_over='T', channel_index=0, z_index=0, t_range=None):
    """
    Calcula proyección media tras escoger canal y eje de proyección.
      - project_over: 'T' o 'Z'
      - channel_index: canal a usar
      - z_index: si project_over='T', fija el plano Z
      - t_range: (inicio, fin_exclusivo) o None para todo T
    Devuelve:
      proj_2d: ndarray (Y, X)
      used_slices: dict con rebanadas usadas
    """
    T, C, Z, Y, X = arr_tczyx.shape

    # Selección de canal
    if channel_index < 0 or channel_index >= C:
        raise IndexError(f"channel_index fuera de rango (0..{C-1})")
    arr = arr_tczyx[:, channel_index]  # (T, Z, Y, X)

    if project_over == 'T':
        # Selecciona Z fijo, luego promedia sobre T dentro del rango
        if z_index < 0 or z_index >= Z:
            raise IndexError(f"z_index fuera de rango (0..{Z-1})")
        arr = arr[:, z_index]  # (T, Y, X)
        if t_range is not None:
            t0, t1 = t_range
            t0 = max(0, t0)
            t1 = min(T, t1)
            if t0 >= t1:
                raise ValueError("t_range inválido: inicio >= fin")
            arr = arr[t0:t1]  # (Tsel, Y, X)
        proj_2d = arr.mean(axis=0)
        used = {'project_over': 'T', 'channel': channel_index, 'z': z_index, 't_range': t_range or (0, T)}
    elif project_over == 'Z':
        # Promedia sobre Z; si hay rango de T, lo aplica primero y luego colapsa T también (media total sobre Z y T)
        if t_range is not None:
            t0, t1 = t_range
            t0 = max(0, t0)
            t1 = min(T, t1)
            if t0 >= t1:
                raise ValueError("t_range inválido: inicio >= fin")
            arr = arr[t0:t1]  # (Tsel, Z, Y, X)
        # Aquí hay dos interpretaciones posibles; por consistencia con tu flujo,
        # mantendremos proyección SOLO sobre Z, y si se dio rango T tomamos
        # la media adicional sobre T (equivale a 2D final). Si prefieres otra, cámbialo.
        arr = arr.mean(axis=0)      # media sobre T -> (Z, Y, X)
        proj_2d = arr.mean(axis=0)  # media sobre Z -> (Y, X)
        used = {'project_over': 'Z', 'channel': channel_index, 't_range': t_range or (0, T)}
    else:
        raise ValueError("project_over debe ser 'T' o 'Z'.")

    return proj_2d, used


def positive_pixel_stats(img2d):
    """
    Calcula estadísticas (media, varianza, CV) sobre píxeles > 0.
    """
    valid = img2d[img2d > 0]
    if valid.size == 0:
        return {'mean': 0.0, 'variance': 0.0, 'cv': np.nan, 'n': 0}
    mean_val = float(np.round(valid.mean(), 3))
    var_val  = float(np.round(valid.var(), 3))
    cv_val   = float(np.round(valid.std() / valid.mean(), 3))
    return {'mean': mean_val, 'variance': var_val, 'cv': cv_val, 'n': int(valid.size)}


In [9]:
# PIPELINE
# ===============
arr_tczyx, sizes = load_ome_tiff_as_tczyx(ome_tif_path)
print(f"Cargado: {os.path.basename(ome_tif_path)}  |  shape TCZYX = {arr_tczyx.shape}  |  sizes = {sizes}")

# Sustracción de fondo
arr_bgsub = subtract_background_and_clip(arr_tczyx, background_value)

# Proyección
proj2d, used = mean_projection(
    arr_bgsub,
    project_over=project_over,
    channel_index=channel_index,
    z_index=z_index,
    t_range=t_range
)

# Estadísticas (solo > 0)
stats = positive_pixel_stats(proj2d)
print(f"Proyección {used} -> stats: mean={stats['mean']}  var={stats['variance']}  cv={stats['cv']}  (n={stats['n']})")

# Visualización
plt.imshow(proj2d, cmap='gray')
ttl = f"Mean projection ({project_over}) | ch={channel_index}"
if project_over == 'T':
    ttl += f" | z={z_index}"
if t_range is not None:
    ttl += f" | T={t_range[0]}–{t_range[1]}"
ttl += f" | bg={background_value}"
plt.title(ttl)
plt.axis('off')
plt.show()

# Guardado opcional
if save_result:
    out_path = os.path.join(main_dir, output_png)
    tifffile.imwrite(  # también puedes guardar como TIFF si lo prefieres
        out_path.replace(".png", ".tif"),
        (proj2d.astype(np.float32)),
        photometric='minisblack'
    )
    plt.imsave(out_path, proj2d, cmap='gray')
    print(f"Guardados: {out_path} y {out_path.replace('.png','.tif')}")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\ADMIN\\Desktop\\ROIs_refinadas\\Sin_NED\\Exp_20250429_Sin_NED\\20250528_M1_cells2'